# Imports

In [1]:
import pandas as pd
import numpy as np
import helper

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option("display.max_columns", 30)

In [3]:
data_players_raw = pd.read_csv(f"Fifa_players/FIFA22_official_data.csv")

# Get data on fifa players

In [4]:
def get_clean_fifa_ratings(year):
    # Read CSV file of a specific year, clean the data and store it in a DataFrame
    data_fifa_players = pd.read_csv(f"Fifa_players/FIFA{year}_official_data.csv")
    data_fifa_players = data_fifa_players.set_index('ID')
    data_fifa_players = data_fifa_players[data_fifa_players["Overall"] >= 55]

    # Clean players' position
    data_fifa_players["Position"] = data_fifa_players["Position"].str.extract(r">(\w+)")
    if "Best Position" in data_fifa_players.columns:
        data_fifa_players['Position'] = data_fifa_players['Best Position'].fillna(value=data_fifa_players['Position'])
    
    # Keep only desired columns
    data_fifa_players = data_fifa_players[['Name', 'Nationality', 'Overall', 'Position', 'Club']].dropna()
    data_fifa_players["Year"] = [year]*len(data_fifa_players)
    data_fifa_players['Club'] = data_fifa_players['Club'].str.upper()
    
    # Remove old players that have a name starting with a number (e.g. "14 D. Beckam")
    data_fifa_players['Name'] = data_fifa_players['Name'].transform(lambda x: np.NaN if (x.lstrip(" ")[0] == "0" or x.lstrip(" ")[0] == "1" or x.lstrip(" ")[0] == "2") else x)
    data_fifa_players = data_fifa_players.dropna()
    
    # We could improve the Overall (rating) with the potential of the player, the age, etc...

    return data_fifa_players

def clean_position(x, data_previous_year):
    # Function applied to a dataframe in the function "get_all_fifa_ratings"
    # Tries to reduce the number of players that have a postion names:
    # SUB for substitute or RES for reserviste (in which case we don't know there
    # position on the field!)
    if x["Position"] not in ["RES", "SUB"]:
        return x
    else:
        try:
            x["Position"] = data_previous_year.loc[x.name]["Position"]
        except KeyError:
            pass
    return x

def get_all_fifa_ratings():
    # Put all the data together (for each of the years) and clean the players' position
    data_all = pd.DataFrame()
    for year in range(17, 24):
        data_year = get_clean_fifa_ratings(year)
        if year >= 18:
            data_year = data_year.apply(clean_position, args=[data_all[data_all["Year"] == (year-1)]], axis=1)
        data_all = pd.concat([data_all, data_year])
    return data_all


In [5]:
data_all_players = get_all_fifa_ratings()

# Get data on games

In [28]:
def match_club_names(x):
    # to be applied to a DataFrame
    # Changes the names of the club to match those in fifa
    if x["home_team"] in helper.DICT_TO_CLUB_NAME_DATA_TO_FIFA.keys():
        x["home_team"] = helper.DICT_TO_CLUB_NAME_DATA_TO_FIFA[x["home_team"]]
    if x["away_team"] in helper.DICT_TO_CLUB_NAME_DATA_TO_FIFA.keys():
        x["away_team"] = helper.DICT_TO_CLUB_NAME_DATA_TO_FIFA[x["away_team"]]
    return x

def get_result(x):
    # to be applied to a DataFrame
    # to add a column mentionning the result of the game
    if x["home_score"] > x["away_score"]:
        return "w_home"
    elif x["home_score"] < x["away_score"]:
        return "w_away"
    return "draw"

def find_team_recent_shape(x, data_games, location, nb_old_games=3):
    # to be applied to a DataFrame
    # Get the result of the teams on the past previous games
    old_games_team = data_games[
        (data_games["home_team"] == x[f"{location}_team"]) +
        (data_games["away_team"] == x[f"{location}_team"])].loc[:x.name].iloc[:-1]
    
    if len(old_games_team) < nb_old_games:
        return x
    else:
        for i in range(1, nb_old_games + 1):
            old_game = old_games_team.iloc[-i]
            if old_game["home_team"] == x[f"{location}_team"]:
                if old_game["Result"] == "w_home":
                    x[f"{location}_{i}_games_ago"] = "win"
                elif x["Result"] == "draw":
                    x[f"{location}_{i}_games_ago"] = "draw"
                else:
                    x[f"{location}_{i}_games_ago"] = "lose"
            else:
                if old_game["Result"] == "w_home":
                    x[f"{location}_{i}_games_ago"] = "lose"
                elif x["Result"] == "draw":
                    x[f"{location}_{i}_games_ago"] = "draw"
                else:
                    x[f"{location}_{i}_games_ago"] = "win"
        return x

def alternative_find_team_recent_shape(x, data_games, location, nb_old_games=3):
    # to be applied to a DataFrame
    # Similar to the previous function but puts the goals difference of the past few games
    # instead of just the result
    old_games_team = data_games[
        (data_games["home_team"] == x[f"{location}_team"]) +
        (data_games["away_team"] == x[f"{location}_team"])].loc[:x.name].iloc[:-1]
    
    if len(old_games_team) < nb_old_games:
        return x
    else:
        for i in range(1, nb_old_games + 1):
            old_game = old_games_team.iloc[-i]
            if old_game["home_team"] == x[f"{location}_team"]:
                x[f"{location}_{i}_games_ago"] = old_game["home_score"] - old_game["away_score"]
            else:
                x[f"{location}_{i}_games_ago"] = old_game["away_score"] - old_game["home_score"]
        return x


def get_clean_games_leagues(list_leagues_to_get=["france", "england"], nb_old_games=3):
    # Gets the data from the games of various leagues
    folder_path = "data_leagues/results/"
    data_games = pd.DataFrame()
    for league in list_leagues_to_get:
        data_games = pd.concat([
            data_games,
            pd.read_csv(folder_path + league + ".csv", index_col='date', parse_dates=True).loc["2016-04":]],
            axis=0)

    data_games = data_games[["home", "away", "gh", "ga", "competition"]].rename(
        columns={
            "home": "home_team",
            "away": "away_team",
            "gh": "home_score",
            "ga": "away_score",
            "competition": "tournament"
        }
    ).reset_index()

    data_games['home_team'] = data_games['home_team'].str.upper()
    data_games['away_team'] = data_games['away_team'].str.upper()
    data_games = data_games.apply(match_club_names, axis=1)

    # Transform the score into win, lose or draw
    data_games["Result"] = data_games.apply(get_result, axis=1)

    # Add shape of the teams on their previous games
    for location in ["home", "away"]:
        for i in range(1, nb_old_games+1):
            data_games[f"{location}_{i}_games_ago"] = np.NaN
    data_games = data_games.apply(alternative_find_team_recent_shape, args=(data_games, "home", nb_old_games), axis=1)
    data_games = data_games.apply(alternative_find_team_recent_shape, args=(data_games, "away", nb_old_games), axis=1)

    data_games['Year'] = data_games["date"].transform(lambda x: int(x.year) - 2000 if x.month < 8 else int(x.year) + 1 - 2000)
    data_games = data_games[data_games['Year'] > 16]
    data_games = data_games.set_index('date')

    
    data_games["home_team_year"] = data_games["home_team"] + "_" + data_games["Year"].astype(str)
    data_games["away_team_year"] = data_games["away_team"] + "_" + data_games["Year"].astype(str)
    
    return data_games

In [29]:
data_games = get_clean_games_leagues(list_leagues_to_get=["france", "england", "spain", "germany", "uefa-cl", "uefa-cw"], nb_old_games=5)

In [8]:
# old_games_team = data_games[
#     (data_games["home_team"] == "AS NANCY LORRAINE") +
#     (data_games["away_team"] == "AS NANCY LORRAINE")].loc[:"2017-04-08"].iloc[:-1]

In [9]:
# data_all_players[data_all_players["Club"].str.contains("BOURNE")]

In [10]:
# TEST: Get all of the potential players in a team
# data_all_players[
#     (data_all_players["Club"] == "RACING CLUB DE LENS") &
#     (data_all_players["Year"] == 20)
# ].sort_values("Overall", ascending=False)

In [11]:
# TEST: Check which teams aren't in the fifa database (their names have
# to be matched with those in fifa)
# def manual_cleaning_names(data_games):

# list_teams_to_replace = []
# for team in data_games["home_team"].unique():
#     if team not in list(data_all_players["Club"]):
#         print(f"Name games : {team}")
#         list_teams_to_replace.append(team)
# print(list_teams_to_replace)

# Put fifa players in their teams

In [12]:
def get_potential_players(players_country, list_fifa_positions):
    potential_players = pd.DataFrame(columns=["ID", "Name", "Overall"])
    for fifa_position in list_fifa_positions:
        players_country_and_position = players_country[players_country["Position"] == fifa_position].sort_values("Overall", ascending=False)
        if len(players_country_and_position) != 0:
            potential_players = pd.concat(
                [
                    potential_players,
                    pd.DataFrame({
                        "ID": [players_country_and_position.iloc[0].name], 
                        "Name": [players_country_and_position.iloc[0]["Name"]],
                        "Overall": [players_country_and_position.iloc[0]["Overall"]]
                    })
                ],
                axis=0)
    return potential_players
    

def get_players_team(team, data_players, year, version, display=False):
    # version: "Club" or "Nationality"
    players_country = data_players[
        (data_players[version] == team) &
        (data_players["Year"] == year)]
    dict_positions = {
        "ST1":["ST", "CF", "LS", "RS"],
        "ST2":["ST", "CF", "LF", "RF", "LS", "RS", "RW", "LW"],
        "CM1":["CM", "LCM", "RCM", "CAM", "RAM", "LAM"],
        "CM2":["CM", "LCM", "RCM", "CDM", "RDM", "LDM"],
        "LM": ["LW", "LM", 'LAM', 'CAM'],
        "RM": ["RM", "RW", "RAM", 'CAM'],
        "CB1":["CB", "LCB", "RCB"],
        "CB2":["CB", "LCB", "RCB"],
        "LB": ["LWB", "LB", "CB", "LCB"],
        "RB": ["RWB", "RB", "CB", "RCB"],
        "GK": ["GK"]
        }
    
    list_overall_players_in_team = []
    for defined_position, list_fifa_positions in dict_positions.items():
        potential_players = get_potential_players(players_country, list_fifa_positions)
            # print(players_country_and_position.iloc[0])
            # print(potential_players)
        if len(potential_players) == 0:
            list_fifa_positions = ["SUB", "RES"]
            potential_players = get_potential_players(players_country, list_fifa_positions)
        if len(potential_players) == 0:
            potential_players = pd.DataFrame({
                "ID": [0], 
                "Name": ["Average"],
                "Overall": [int(np.mean(list_overall_players_in_team)*0.95)]
            })
        else:
            potential_players = potential_players.sort_values("Overall", ascending=False)
            players_country = players_country.drop(potential_players.iloc[0].ID, axis=0)
        list_overall_players_in_team.append(potential_players.iloc[0]["Overall"])
        if display:
            print(f"{defined_position}: - {list_fifa_positions} - {potential_players.iloc[0]['Name']} - {potential_players.iloc[0]['Overall']}")

    players_in_team = pd.DataFrame(columns=["Year", "Team"]+list(dict_positions.keys()))
    players_in_team.loc[len(players_in_team)] = [year, team] + list_overall_players_in_team
        
    return players_in_team


In [13]:
# TEST: get players of a specific team
# get_players_team("FC BAYERN MÜNCHEN", data_all_players, year=23, display=True, version="Club")

In [14]:
def get_roster(data_games, data_all_players, version):
    # get the roster of every team in the games data
    data_rosters = pd.DataFrame()
    teams_not_found = []
    for year in data_all_players["Year"].unique():
        for team in (pd.concat([data_games[data_games["Year"] == year]["home_team"], data_games[data_games["Year"] == year]["away_team"]])).unique():
            try:
                data_rosters = pd.concat([data_rosters, get_players_team(team, data_all_players, year=year, version=version)], axis=0)
            except ValueError:
                teams_not_found.append(f"{year}_{team}")
                print(year, team)
    data_rosters["Team_year"] = data_rosters["Team"] + "_" + data_rosters["Year"].astype(str)
    # print(teams_not_found)
    return data_rosters

In [15]:
data_rosters = get_roster(data_games, data_all_players, version="Club")

/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_

17 QARABAG FK
17 APOEL NIKOSIA
17 DINAMO TBILISI
17 RED STAR BELGRADE
17 DUNDALK FC
17 FC KOEBENHAVN
17 STEAUA BUCURESTI


/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


17 HAPOEL BEER SHEVA


/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_

...

/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


19 VIKTORIA PLZEN


/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_

19 CSKA MOSKVA
19 AS ROMA
19 LOKOMOTIV MOSKVA
19 SSC NAPOLI
19 KS PERPARIMI KUKES
19 CFR CLUJ
19 PFC LUDOGORETS RAZGRAD


/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thibaud/miniconda3/envs/sdse/lib/python3.9/site-packages/numpy/core/_

In [16]:
data_rosters

,Year,Team,ST1,ST2,CM1,CM2,LM,RM,CB1,CB2,LB,RB,GK,Team_year
0,17,SPORTING CLUB DE BASTIA,73,70,74,73,70,75,74,74,73,73,78,SPORTING CLUB DE BASTIA_17
0,17,AS MONACO,82,80,83,82,80,77,84,78,77,79,84,AS MONACO_17
0,17,FC GIRONDINS DE BORDEAUX,77,75,78,80,77,75,76,74,73,76,81,FC GIRONDINS DE BORDEAUX_17
0,17,MONTPELLIER HSC,76,74,80,73,75,77,79,75,75,73,75,MONTPELLIER HSC_17
0,17,DIJON FCO,74,71,74,73,73,72,71,70,71,72,76,DIJON FCO_17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,19,INTER,87,82,81,85,81,85,86,84,76,81,88,INTER_19
0,19,CLUB BRUGGE KV,78,76,79,78,74,75,76,76,74,74,71,CLUB BRUGGE KV_19
0,19,SHAKHTAR DONETSK,76,74,82,80,82,74,75,71,70,75,77,SHAKHTAR DONETSK_19
0,19,JUVENTUS,94,85,89,86,85,84,90,86,86,83,86,JUVENTUS_19


In [30]:
data_full = pd.merge(left=data_games, right=data_rosters, how='inner', left_on="home_team_year", right_on="Team_year")
data_full = pd.merge(left=data_full, right=data_rosters, how='inner', left_on="away_team_year", right_on="Team_year")
data_full.drop(["Year_x", "tournament", "Year_y", "home_team", "away_team", "home_team_year", "away_team_year", "Team_year_x", "Team_year_y", "Team_x", "Team_y", "Year"], axis=1, inplace=True)
data_full = data_full.dropna()

data_full.drop(["home_score", "away_score"], axis=1, inplace=True)
data_full

,Result,home_1_games_ago,home_2_games_ago,home_3_games_ago,home_4_games_ago,home_5_games_ago,away_1_games_ago,away_2_games_ago,away_3_games_ago,away_4_games_ago,away_5_games_ago,ST1_x,ST2_x,CM1_x,CM2_x,...,CB2_x,LB_x,RB_x,GK_x,ST1_y,ST2_y,CM1_y,CM2_y,LM_y,RM_y,CB1_y,CB2_y,LB_y,RB_y,GK_y
0,w_away,1.00,1.00,0.00,-1.00,-1.00,4.00,0.00,4.00,4.00,6.00,73,70,74,73,...,74,73,73,78,86,72,86,85,86,83,89,83,82,82,82
1,w_home,1.00,0.00,2.00,-5.00,1.00,3.00,1.00,4.00,0.00,4.00,82,80,83,82,...,78,77,79,84,86,72,86,85,86,83,89,83,82,82,82
2,w_away,4.00,0.00,2.00,1.00,0.00,1.00,2.00,0.00,2.00,1.00,77,75,78,80,...,74,73,76,81,86,72,86,85,86,83,89,83,82,82,82
3,w_home,-1.00,0.00,0.00,2.00,0.00,2.00,1.00,2.00,4.00,1.00,76,74,80,73,...,75,75,73,75,86,72,86,85,86,83,89,83,82,82,82
4,w_away,1.00,0.00,0.00,2.00,-1.00,0.00,2.00,1.00,5.00,-1.00,74,71,74,73,...,70,71,72,76,86,72,86,85,86,83,89,83,82,82,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,w_away,-4.00,-4.00,-2.00,1.00,-2.00,0.00,6.00,1.00,3.00,-2.00,78,73,79,79,...,76,75,75,69,82,82,76,79,80,76,79,78,76,79,82
4533,draw,3.00,4.00,-1.00,1.00,-1.00,0.00,0.00,2.00,-3.00,0.00,84,81,83,79,...,77,80,78,83,82,82,76,79,80,76,79,78,76,79,82
4534,w_away,1.00,-1.00,-2.00,-4.00,-3.00,3.00,1.00,3.00,1.00,3.00,79,78,79,79,...,80,75,75,82,82,82,76,79,80,76,79,78,76,79,82
4535,w_home,2.00,2.00,0.00,2.00,0.00,0.00,-1.00,1.00,-2.00,3.00,90,83,87,84,...,84,84,86,88,78,77,84,80,76,82,82,80,82,79,79


In [42]:
data_full.drop(["away_1_games_ago", "away_2_games_ago", "away_3_games_ago", "away_4_games_ago", "away_5_games_ago"], axis=1)

,Result,home_1_games_ago,home_2_games_ago,home_3_games_ago,home_4_games_ago,home_5_games_ago,ST1_x,ST2_x,CM1_x,CM2_x,LM_x,RM_x,CB1_x,CB2_x,LB_x,RB_x,GK_x,ST1_y,ST2_y,CM1_y,CM2_y,LM_y,RM_y,CB1_y,CB2_y,LB_y,RB_y,GK_y
0,w_away,1.00,1.00,0.00,-1.00,-1.00,73,70,74,73,70,75,74,74,73,73,78,86,72,86,85,86,83,89,83,82,82,82
1,w_home,1.00,0.00,2.00,-5.00,1.00,82,80,83,82,80,77,84,78,77,79,84,86,72,86,85,86,83,89,83,82,82,82
2,w_away,4.00,0.00,2.00,1.00,0.00,77,75,78,80,77,75,76,74,73,76,81,86,72,86,85,86,83,89,83,82,82,82
3,w_home,-1.00,0.00,0.00,2.00,0.00,76,74,80,73,75,77,79,75,75,73,75,86,72,86,85,86,83,89,83,82,82,82
4,w_away,1.00,0.00,0.00,2.00,-1.00,74,71,74,73,73,72,71,70,71,72,76,86,72,86,85,86,83,89,83,82,82,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,w_away,-4.00,-4.00,-2.00,1.00,-2.00,78,73,79,79,76,76,76,76,75,75,69,82,82,76,79,80,76,79,78,76,79,82
4533,draw,3.00,4.00,-1.00,1.00,-1.00,84,81,83,79,82,78,80,77,80,78,83,82,82,76,79,80,76,79,78,76,79,82
4534,w_away,1.00,-1.00,-2.00,-4.00,-3.00,79,78,79,79,77,79,81,80,75,75,82,82,82,76,79,80,76,79,78,76,79,82
4535,w_home,2.00,2.00,0.00,2.00,0.00,90,83,87,84,87,86,88,84,84,86,88,78,77,84,80,76,82,82,80,82,79,79


In [18]:
data_full.to_csv("data_full_national_leagues.csv", index=False)

# Predictions

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [43]:
X_train, X_test, Y_train, Y_test = train_test_split(data_full.drop("Result", axis=1), data_full["Result"], test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
# We can change the multi_class
model = LogisticRegression(multi_class="multinomial", max_iter=5000)

In [45]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=5000, multi_class='multinomial')

In [46]:
model.score(X_train, Y_train), model.score(X_test, Y_test)

(0.5409323479249574, 0.5545454545454546)

In [35]:
from lazypredict.Supervised import LazyClassifier

In [47]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, Y_train, Y_test)
models.sort_values("Accuracy", ascending=False)

100%|██████████| 29/29 [00:46<00:00,  1.61s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.57,0.49,None,0.52,1.85
LogisticRegression,0.55,0.47,None,0.50,0.21
LinearSVC,0.55,0.46,None,0.49,3.99
RidgeClassifier,0.55,0.46,None,0.49,0.08
SVC,0.55,0.46,None,0.47,3.20
RidgeClassifierCV,0.55,0.46,None,0.49,0.10
LinearDiscriminantAnalysis,0.55,0.47,None,0.50,0.13
CalibratedClassifierCV,0.55,0.45,None,0.47,15.85
RandomForestClassifier,0.54,0.47,None,0.51,2.42


# World cup predictions

In [26]:
def predict_game(team_home, team_away, year, model):
    x = pd.merge(get_players_team(team_home, data_all_players, year=year, display=True, version="Club"), get_players_team(team_away, data_all_players, year=year, display=True, version="Club"), on="Year")
    x = x.drop(["Year", "Team_x", "Team_y"], axis=1)
    print(['draw', 'w_away', 'w_home'])
    return model.predict_proba(x)

In [27]:
predict_game("AS MONACO","PARIS SAINT-GERMAIN",  18, model)

ST1: - ['ST', 'CF', 'LS', 'RS'] -  Falcao - 85
ST2: - ['ST', 'CF', 'LF', 'RF', 'LS', 'RS', 'RW', 'LW'] -  S. Jovetić - 80
CM1: - ['CM', 'LCM', 'RCM', 'CAM', 'RAM', 'LAM'] -  T. Lemar - 83
CM2: - ['CM', 'LCM', 'RCM', 'CDM', 'RDM', 'LDM'] -  Fabinho - 84
LM: - ['LW', 'LM', 'LAM', 'CAM'] -  Rony Lopes - 79
RM: - ['RM', 'RW', 'RAM', 'CAM'] -  K. Baldé - 81
CB1: - ['CB', 'LCB', 'RCB'] -  K. Glik - 84
CB2: - ['CB', 'LCB', 'RCB'] -  Jemerson - 80
LB: - ['LWB', 'LB', 'CB', 'LCB'] -  A. Raggi - 76
RB: - ['RWB', 'RB', 'CB', 'RCB'] -  D. Sidibé - 79
GK: - ['GK'] -  D. Subašić - 84
ST1: - ['ST', 'CF', 'LS', 'RS'] -  E. Cavani - 88
ST2: - ['ST', 'CF', 'LF', 'RF', 'LS', 'RS', 'RW', 'LW'] -  K. Mbappé - 84
CM1: - ['CM', 'LCM', 'RCM', 'CAM', 'RAM', 'LAM'] -  Neymar - 92
CM2: - ['CM', 'LCM', 'RCM', 'CDM', 'RDM', 'LDM'] -  M. Verratti - 87
LM: - ['LW', 'LM', 'LAM', 'CAM'] -  J. Draxler - 84
RM: - ['RM', 'RW', 'RAM', 'CAM'] -  A. Di María - 84
CB1: - ['CB', 'LCB', 'RCB'] -  Thiago Silva - 87
CB2: - ['CB'

array([[2.06054876e-06, 9.93132894e-01, 6.86504582e-03]])